<a href="https://colab.research.google.com/github/minaR0404/Kaggle_Stable-Diffusion-Image-to-Prompts/blob/main/CLIP_ViT_L_14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Mon Apr 24 17:05:39 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install kaggle

from google.colab import drive
drive.mount('/content/drive')

import os
import json
f = open("/content/drive/MyDrive/kaggle.json", 'r')
json_data = json.load(f)
os.environ['KAGGLE_USERNAME'] = json_data['username']
os.environ['KAGGLE_KEY'] = json_data['key']
!kaggle competitions download -c stable-diffusion-image-to-prompts
!unzip '/content/stable-diffusion-image-to-prompts.zip'

!mkdir -p /kaggle/input/

!kaggle datasets download -d inversion/sentence-transformers-222
!unzip sentence-transformers-222.zip -d sentence-transformers-222
!kaggle datasets download -d leonidkulyk/clip-interrogator-wheels-x
!unzip clip-interrogator-wheels-x.zip -d clip-interrogator-wheels-x
# !kaggle datasets download -d leonidkulyk/clip-interrogator-models-x
# !unzip clip-interrogator-models-x.zip -d clip-interrogator-models-x

!pip install git+https://github.com/openai/CLIP.git

# !kaggle kernels output minar44/data-similarity -p /path/to/dest
# !unzip -j /content/drive/MyDrive/Colab\ Notebooks/diffusion-data-384/archive2.zip -d /kaggle/input/DiffusionDB-2M-resize384
!kaggle kernels output minar44/data-similarity -p /path/to/dest
!unzip -j /content/drive/MyDrive/Colab\ Notebooks/diffusion-data/500k.zip -d /kaggle/input/DiffusionDB-500k-resize224

!pip -qqq install sentence-transformers

!pip install torch==1.13.0+cu117 torchvision==0.14.0+cu117 torchaudio===0.13.0+cu117 -f https://downlo

In [ ]:
!pip install transformers==4.12.0

wheels_path = "/content/clip-interrogator-wheels-x"
clip_interrogator_whl_path = f"{wheels_path}/clip_interrogator-0.4.3-py3-none-any.whl"
!pip install --no-index --find-links $wheels_path $clip_interrogator_whl_path -q

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 45.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 88.3 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895259 sha256=1b5b8d7d6be18a27a0b028f775306a5670136010c5e08078ad22fe0883e051f4
  Stored in directory: /root/.cache/pip/wheels/12/1c/3d/46cf06718d63a32ff798a89594b61e7f345ab6b36d909ce033
Successfully built sacremoses
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.28.1
    Uninstalling transformers-4.28.1:
      Successfully uninst

In [ ]:
import sys
import numpy as np
import pickle
import pandas as pd
from tqdm.notebook import tqdm
import torch
from sklearn.model_selection import train_test_split
from glob import glob
from torch import nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from pathlib import Path
import re
import open_clip

sys.path.append('/content/sentence-transformers-222/sentence-transformers')
from transformers import AutoModel, AutoProcessor
from transformers import BertTokenizer
from sentence_transformers import SentenceTransformer, models

device = torch.device(f'cuda:0' if torch.cuda.is_available() else 'cpu')
st_model = SentenceTransformer('/content/sentence-transformers-222/all-MiniLM-L6-v2')
st_model = st_model.to(device)

clip_processor = AutoProcessor.from_pretrained("openai/clip-vit-large-patch14")
BATCHSIZE = 12
SAVE_OPT_CKP = True
SAVE_MODEL_CKP = True
UNFREEZE_START = 18 # set it to lower number when significantly more samples are included.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn

run_name = f'ViT-L-14_OpenAI_f18'

Downloading:   0%|          | 0.00/316 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.41k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/939k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/512k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/389 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/905 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12M [00:00<?, ?B/s]

In [ ]:
input_size = 224
transform = transforms.Compose([
        transforms.Resize(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])

In [ ]:
def cosine_similarity_loss(pred, target):
    cos = nn.CosineSimilarity(dim=1)
    output = -cos(pred, target).mean()
    return output

def get_train_test_split():
    """add your image paths and embedding labels here"""
    # df = pd.read_csv('/path/to/dest/df_sim0.70.csv')
    # df['filepath'] = df['filepath'].apply(lambda x: re.sub(r'/kaggle/input/diffusiondb-2m-part-\d{4}-to-\d{4}-of-2000', '/kaggle/input/DiffusionDB-2M-resize384', x).replace('.png', '.jpeg'))
    df = pd.read_csv('/path/to/dest/df512_sim0.90.csv')
    df['filepath'] = df['filepath'].apply(lambda x: re.sub(r'/kaggle/input/diffusiondb-2m-part-\d{4}-to-\d{4}-of-2000', '/kaggle/input/DiffusionDB-500k-resize224', x).replace('.png', '.jpeg'))
    # df = df[:2000]
    max_length = 77  # 制限する最大文字数
    df["prompt"] = df["prompt"].str.slice(stop=max_length)
    trn_df, val_df = train_test_split(df, test_size=0.1, random_state=42)
    trn_ceil = len(trn_df) % BATCHSIZE
    val_ceil = len(val_df) % BATCHSIZE
    trn_df = trn_df[:len(trn_df)-trn_ceil].reset_index(drop=True)
    val_df = val_df[:len(val_df)-val_ceil].reset_index(drop=True)

    train_images = trn_df['filepath'].to_list()
    test_images = val_df['filepath'].to_list()
    train_prompt = trn_df['prompt'].to_list()
    test_prompt = val_df['prompt'].to_list()

    return train_images, test_images, train_prompt, test_prompt


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        clip = AutoModel.from_pretrained("openai/clip-vit-large-patch14")

        self.visual = clip.vision_model
        self.fc = nn.Linear(1024, 384)

    def forward(self, x):
        out = self.visual(x)['pooler_output']
        return self.fc(out)


def load_pretrained_model():
    model = Net()

    trainable_model_weights = False
    for name, child in model.named_children():
        if name == 'visual':  # 'vision'
            for pn, p in child.named_parameters():
                if str(UNFREEZE_START) in pn:
                    """start unfreezing layer , the weights are trainable"""
                    trainable_model_weights = True
                p.requires_grad = trainable_model_weights
                if p.requires_grad:
                    print(f"{pn} is set to be trainable.")

    return model.to(device)

class DiffusionCollator:
    def __call__(self, batch):
        images, prompt = zip(*batch)
        images = torch.stack(images)
        targets = st_model.encode(
            prompt,
            show_progress_bar=False,
            convert_to_tensor=True
        )
        return images, targets

class IMGDataset:
    def __init__(self, image_paths, prompt, transform=transform, clip_processor=clip_processor):
        self.images = image_paths
        self.prompt = prompt
        self.transform = transform
        self.input_processor = clip_processor

    def __len__(self):
        return len(self.images)

    def __getitem__(self, item):
        image = Image.open(self.images[item]).convert("RGB")
        image = self.transform(image)
        image = (image - image.min()) / (image.max() - image.min())
        prompt = self.prompt[item]
        image = self.input_processor(prompt, image, return_tensors="pt")["pixel_values"]
        return image, prompt


if __name__ == "__main__":
    """main training"""
    Path(f"../{run_name}").mkdir(exist_ok=True)

    NEPOCH = 3
    BestEpoch = 0
    BestSim = 0
    train_images, test_images, train_prompt, test_prompt = get_train_test_split()

    print(f"test size: {len(test_images)}, train size: {len(train_images)}")

    nn_model = load_pretrained_model()
#     nn_model = torch.compile(nn_model)
    optimizer = optim.AdamW(filter(lambda p: p.requires_grad, nn_model.parameters()), lr=1e-4)
    optimizer.zero_grad()

    collator = DiffusionCollator()
    test_dataloader = DataLoader(dataset=IMGDataset(test_images, test_prompt),
                                 batch_size=BATCHSIZE, shuffle=False, num_workers=0, pin_memory=False, collate_fn=collator)
    train_dataloader = DataLoader(dataset=IMGDataset(train_images, train_prompt),
                                 batch_size=BATCHSIZE, shuffle=True, num_workers=0, pin_memory=False, collate_fn=collator)

    for epoch in range(NEPOCH):
        epoch_loss = 0
        for s, batch_data in enumerate(tqdm(train_dataloader)):
            batch_images, batch_targets = batch_data
            batch_images = np.reshape(batch_images, (BATCHSIZE, 3, 224, 224))
            batch_images, batch_targets = batch_images.to(device), batch_targets.to(device)
            pred = nn_model(batch_images)
            cosine_loss = cosine_similarity_loss(pred, batch_targets)
            loss = cosine_loss
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            epoch_loss += -cosine_loss.item()
        epoch_loss /= len(train_dataloader)
        print(f"epoch: {epoch}, training loss: {epoch_loss}")

        """test loss"""
        epoch_loss = 0
        with torch.no_grad():
            for batch_images, batch_targets in tqdm(test_dataloader):
                batch_images = np.reshape(batch_images, (BATCHSIZE, 3, 224, 224))
                batch_images, batch_targets = batch_images.to(device), batch_targets.to(device)
                pred = nn_model(batch_images)
                loss = -cosine_similarity_loss(pred, batch_targets)
                epoch_loss += loss.item()
            epoch_loss /= len(test_dataloader)
        print(f"epoch: {epoch}, test loss: {epoch_loss}")

        if epoch_loss > BestSim:
            BestSim = epoch_loss
            BestEpoch = epoch + 1
            print(f"save best model at {BestSim} with epoch {BestEpoch}")
            if SAVE_MODEL_CKP:
                torch.save(nn_model.state_dict(), f"/content/drive/My Drive/Colab Notebooks/ViT-H-14/{run_name}_{NEPOCH}epoch.pt")
            if SAVE_OPT_CKP:
                torch.save(optimizer.state_dict(), f"/content/drive/My Drive/Colab Notebooks/ViT-H-14/{run_name}_{NEPOCH}epoch_opt.pt")

        if epoch - 3 > BestEpoch:
            print(f"early stop at {epoch+1} with best epoch {BestEpoch} and test similarity {BestSim}.")
            break

test size: 46272, train size: 416704


Downloading:   0%|          | 0.00/1.59G [00:00<?, ?B/s]

encoder.layers.18.self_attn.k_proj.weight is set to be trainable.
encoder.layers.18.self_attn.k_proj.bias is set to be trainable.
encoder.layers.18.self_attn.v_proj.weight is set to be trainable.
encoder.layers.18.self_attn.v_proj.bias is set to be trainable.
encoder.layers.18.self_attn.q_proj.weight is set to be trainable.
encoder.layers.18.self_attn.q_proj.bias is set to be trainable.
encoder.layers.18.self_attn.out_proj.weight is set to be trainable.
encoder.layers.18.self_attn.out_proj.bias is set to be trainable.
encoder.layers.18.layer_norm1.weight is set to be trainable.
encoder.layers.18.layer_norm1.bias is set to be trainable.
encoder.layers.18.mlp.fc1.weight is set to be trainable.
encoder.layers.18.mlp.fc1.bias is set to be trainable.
encoder.layers.18.mlp.fc2.weight is set to be trainable.
encoder.layers.18.mlp.fc2.bias is set to be trainable.
encoder.layers.18.layer_norm2.weight is set to be trainable.
encoder.layers.18.layer_norm2.bias is set to be trainable.
encoder.laye

  0%|          | 0/6511 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (78 > 77). Running this sequence through the model will result in indexing errors


...

epoch: 0, test loss: 0.6180619183582554
save best model at 0.6180619183582554 with epoch 1


  0%|          | 0/6511 [00:00<?, ?it/s]

epoch: 1, training loss: 0.6402942713386103


  0%|          | 0/723 [00:00<?, ?it/s]

epoch: 1, test loss: 0.6280672140471009
save best model at 0.6280672140471009 with epoch 2
